In [1]:
import pandas as pd
import numpy as np
from untrade.client import Client

ModuleNotFoundError: No module named 'untrade'

In [ ]:
def process_data(df):
    """
    Process the input dataframe to calculate indicators, generate trade signals,
    and apply stop-loss (SL) and take-profit (TP) logic based on the new strategy.

    Parameters:
    - df (DataFrame): Input dataframe with OHLC and volume data.

    Returns:
    - df (DataFrame): Processed dataframe with signals, trade types, and additional columns.
    """
    df = df.copy()
    
    # Calculate moving averages
    df['MA_13'] = df['close'].rolling(window=13).mean()
    df['MA_15'] = df['close'].rolling(window=15).mean()
    df['MA_200'] = df['close'].rolling(window=200).mean()
    
    # Calculate Bollinger Bands (20-period SMA with 2 standard deviations)
    df['BB_Mid'] = df['close'].rolling(window=20).mean()
    df['BB_Upper'] = df['BB_Mid'] + (2 * df['close'].rolling(window=20).std())
    df['BB_Lower'] = df['BB_Mid'] - (2 * df['close'].rolling(window=20).std())
    
    # Calculate average volume over the last 26 periods
    df['Avg_Volume'] = df['volume'].rolling(window=26).mean()
    
    # Initialize trade columns
    df['signals'] = 0  # Buy (1), Sell (-1), Square-off (0)
    df['trade'] = 0  # Track active trades (1 for long, -1 for short)
    df['SL'] = np.nan  # Stop-loss levels
    df['TP'] = np.nan  # Take-profit levels
    df['trade_type'] = 'square-off'  # Trade type ('long', 'short', 'square-off')
    
    # Loop through data to calculate signals and update trade states
    for i in range(200, len(df)):
        curr_close = df['close'].iloc[i]
        curr_low = df['low'].iloc[i]
        curr_high = df['high'].iloc[i]
        curr_vol = df['volume'].iloc[i]
        avg_vol = df['Avg_Volume'].iloc[i]
        prev_trade = df['trade'].iloc[i-1]
        
        # Check if volume condition is met
        vol_condition = curr_vol >= 1.3 * avg_vol
        
        # Long condition
        if df['MA_13'].iloc[i] > df['MA_15'].iloc[i] and curr_close > df['MA_200'].iloc[i] and vol_condition:
            if prev_trade == -1:  # Square-off short trade
                df.loc[df.index[i], 'signals'] = -1
                df.loc[df.index[i], 'trade'] = 0
                df.loc[df.index[i], 'trade_type'] = 'square-off'
            else:  # Open long trade
                delta = curr_close - curr_low
                df.loc[df.index[i], 'signals'] = 1
                df.loc[df.index[i], 'SL'] = curr_low - (0.1 * delta)
                df.loc[df.index[i], 'TP'] = df['BB_Upper'].iloc[i]
                df.loc[df.index[i], 'trade'] = 1
                df.loc[df.index[i], 'trade_type'] = 'long'
        
        # Short condition
        elif df['MA_13'].iloc[i] < df['MA_15'].iloc[i] and curr_close < df['MA_200'].iloc[i] and vol_condition:
            if prev_trade == 1:  # Square-off long trade
                df.loc[df.index[i], 'signals'] = 1
                df.loc[df.index[i], 'trade'] = 0
                df.loc[df.index[i], 'trade_type'] = 'square-off'
            else:  # Open short trade
                delta = curr_high - curr_close
                df.loc[df.index[i], 'signals'] = -1
                df.loc[df.index[i], 'SL'] = curr_high + (0.1 * delta)
                df.loc[df.index[i], 'TP'] = df['BB_Lower'].iloc[i]
                df.loc[df.index[i], 'trade'] = -1
                df.loc[df.index[i], 'trade_type'] = 'short'
        else:
            df.loc[df.index[i], 'trade'] = prev_trade  # Maintain trade state
    
    # Keep only required columns for backtesting
    df = df[['datetime', 'open', 'high', 'low', 'close', 'volume', 'signals', 'trade_type', 
         'SL', 'TP', 'MA_13', 'MA_15', 'MA_200', 'BB_Upper', 'BB_Lower', 'BB_Mid']]
    
    
    return df

def strat(data):
    """
    Adjust leverage dynamically and improve entry price logic.

    Parameters:
    - data (DataFrame): Dataframe with trade signals.

    Returns:
    - data (DataFrame): Processed dataframe with leverage adjustments and optimized entry prices.
    """
    data = data.copy()

    # Ensure signals are not repeated
    signal = []
    prev = None
    for value in data["signals"]:
        if value == prev:
            signal.append(0)  # No action if same signal as previous
        else:
            signal.append(value)  # Keep the signal if different
        prev = value
    data["signals"] = signal

    # Trend Strength Factor (Distance from MA200)
    data["distance_MA200"] = abs(data["close"] - data["MA_200"]) / data["MA_200"]

    # Volatility Factor (Bollinger Band Width)
    data["BB_Width"] = (data["BB_Upper"] - data["BB_Lower"]) / data["BB_Mid"]

    # ATR for Dynamic Entry Price
    data["ATR"] = data["high"].rolling(14).max() - data["low"].rolling(14).min()

    # Win Rate Tracking (Rolling % of Profitable Trades Over Last 10 Trades)
    data["win_rate"] = data["signals"].rolling(10).apply(lambda x: np.mean(x[x > 0]) if np.any(x > 0) else 0, raw=True)

    # Leverage Adjustment Rules
    conditions = [
        (data["distance_MA200"] > 0.05) & (data["BB_Width"] < data["BB_Width"].quantile(0.7)) & (data["win_rate"] > 0.5),  # Strong trend, low volatility, good win rate
        (data["distance_MA200"] > 0.05) & (data["BB_Width"] > data["BB_Width"].quantile(0.7)),  # Strong trend, high volatility
        (data["distance_MA200"] < 0.05),  # Weak trend (near MA200)
        (data["win_rate"] < 0.3)  # Losing streak (reduce risk)
    ]
    leverage_values = [10, 7, 3, 2]  # Leverage values for each condition
    data["leverage"] = np.select(conditions, leverage_values, default=5)

    # Optimized Entry Price Logic
    entry_prices = []
    for i in range(len(data)):
        if data["signals"].iloc[i] == 1:  # Long trade
            entry_price = min(data["BB_Lower"].iloc[i] + 0.5 * data["ATR"].iloc[i], data["close"].iloc[i])
        elif data["signals"].iloc[i] == -1:  # Short trade
            entry_price = max(data["BB_Upper"].iloc[i] - 0.5 * data["ATR"].iloc[i], data["close"].iloc[i])
        else:
            entry_price = data["close"].iloc[i]  # No trade, keep normal price
        entry_prices.append(entry_price)

    data["optimized_entry"] = entry_prices

    # Retain only necessary columns
    data = data[['datetime', 'open', 'high', 'low', 'close', 'volume', 'signals', 'trade_type', 'leverage', 'optimized_entry']].copy()

    return data


def perform_backtest(csv_file_path):
    """
    Perform backtesting using the Untrade SDK.

    Parameters:
    - csv_file_path (str): Path to the CSV file with processed data.

    Returns:
    - result (generator): Backtest results generator.
    """
    # Create Untrade client instance
    client = Client()

    # Perform backtest using Untrade
    result = client.backtest(
        jupyter_id="team64_zelta_hpps",  # User's Jupyter ID for Untrade
        file_path=csv_file_path,
        # leverage=5,  # Set leverage for backtest
        # position=50,  # Set initial position sizeq
    )

    return result

# Load BTC Data
data = pd.read_csv("./ETHUSDT_4h.csv")

# Process data
res = process_data(data)

# Apply filtering strategy to remove duplicate signals
res = strat(res)

# Save processed data to CSV for backtesting
res.to_csv("processed_data_BTC.csv", index=False)

# Perform backtesting
csv_file_path = "processed_data_BTC.csv"
backtest_result = perform_backtest(csv_file_path)

# Print backtest results
for value in backtest_result:
    print(value)